In [ ]:
# Import relevant Python libraries here
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Load dataset into notebook
df = pd.read_csv('Marriage Perception (Responses) - Form Responses 1.csv')

# Shape of dataset
print(f"Size of Dataset (Rows, Cols): {df.shape}\n")

# Peek at head of dataset
print("Preview: ")
df.head()

In [ ]:
# Check for duplicates
print(f"Duplicates:  {df.duplicated().sum()}\n")

# Check for types and missing entries
df.info()